In [10]:
from sklearn.model_selection import train_test_split
import tensorflow as tf



In [11]:
import sys
sys.path.insert(1, '../Pre Processing Stage/')
sys.path.insert(1, '../Processing Stage/HoG')


from commonfunctions import *
from main import *
from hog import *

In [12]:
final_hog=np.load('./numpyArrays/final_hog.npy' )
targets_hog=np.load('./numpyArrays/targets_hog.npy')



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from tensorflow import keras

In [14]:
classifiers = {
    'SVM': SVC(random_state=40, C=1),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'rnd': RandomForestClassifier(random_state=40),
    'voting_clf_soft' : VotingClassifier(
    estimators=[
        ('KNN', KNeighborsClassifier(n_neighbors=7)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42, probability=True, C=1))
    ],
    voting='soft'), 
    'voting_clf_hard': VotingClassifier(
    estimators=[
            ('KNN', KNeighborsClassifier(n_neighbors=7)),
            ('rf', RandomForestClassifier(random_state=42)),
            ('svc', SVC(random_state=42,C=1))
        ],
    voting='hard'),
    
}

In [15]:
from sklearn.metrics import confusion_matrix
import copy


def train_model(x,y,testSize):
    models=[]

    X_train, X_test, y_train, y_test=train_test_split(x, y,test_size=testSize,stratify=y)

    for model_name, model in copy.deepcopy(classifiers).items():
        
        print('############## Training', model_name, "##############")
        model.fit(X_train, y_train)

        accuracy = model.score(X_test, y_test)

        models.append({model_name:model})

        print(model_name, 'accuracy:', accuracy*100, '%')
        y_pred = model.predict(X_test)

        mat = confusion_matrix(y_test, y_pred)

        print(mat)
            

    return models

In [16]:
models_hog=train_model(final_hog,targets_hog,0.2)

############## Training SVM ##############
SVM accuracy: 100.0 %
[[17  0]
 [ 0 21]]
############## Training KNN ##############
KNN accuracy: 100.0 %
[[17  0]
 [ 0 21]]
############## Training rnd ##############
rnd accuracy: 100.0 %
[[17  0]
 [ 0 21]]
############## Training voting_clf_soft ##############
voting_clf_soft accuracy: 100.0 %
[[17  0]
 [ 0 21]]
############## Training voting_clf_hard ##############
voting_clf_hard accuracy: 100.0 %
[[17  0]
 [ 0 21]]


In [17]:
import pickle

def save_models(modelsArray,suffix):
    for model in modelsArray:
        pickle.dump(next(iter(model.values())), open(
            f"./models/{next(iter(model.keys()))}_{suffix}.pkl", "wb"))

In [18]:
save_models(models_hog,"hog")